In [2]:
from langfuse import Langfuse
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from application3 import Application3

In [3]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_claude = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
builder = Application3(m_claude, compiler)

In [5]:
prepare_res = builder.prepare_bot(["bot that greets me by name"], bot_id="1337_420", capabilities=[])

Processing transition: [] FsmState.TYPESPEC
Processing transition: [<FsmState.TYPESPEC: 'typespec'>] FsmState.WAIT
{'user_requests': ['bot that greets me by name'], 'typespec_schema': <fsm_core.typespec.Success object at 0xffff9b566630>}


In [7]:
prepare_res

{'capabilities': [], 'typespec': <fsm_core.typespec.Success at 0xffff9b566630>}

In [9]:
prepare_res["typespec"].typespec

'model UserNameInfo {\n    name: string;\n}\n\nmodel GreetingOptions {\n    style?: string;\n    timeOfDay?: string;\n}\n\nmodel UpdateNameOptions {\n    currentName: string;\n    newName: string;\n}\n\ninterface GreetingBot {\n    @scenario(\n    """\n    Scenario: Saving a new user name\n    When user says "My name is John"\n    Then system should extract the name "John"\n    And store it for future greetings\n    Examples:\n        | Input                       | Extracted Name |\n        | "I am Sarah"                | "Sarah"        |\n        | "Please call me Mike"       | "Mike"         |\n        | "The name\'s Bond, James Bond" | "James Bond"  |\n    """)\n    @llm_func("Save user name for personalized greetings")\n    saveUserName(options: UserNameInfo): void;\n\n    @scenario(\n    """\n    Scenario: Greeting user with stored name\n    When user requests a greeting\n    Then system should respond with personalized greeting using stored name\n    Examples:\n        | Stored 

In [13]:
reasoning = prepare_res["typespec"].reasoning
typespec_schema = prepare_res["typespec"].typespec
fake_msg = f"<reasoning>{reasoning}</reasoning><typespec>{typespec_schema}</typespec>"

In [14]:
final_res = builder.update_bot(
    typespec_schema=fake_msg,
    bot_id="1337_420",
    capabilities=[],
)

Processing transition: [] FsmState.DRIZZLE
Processing transition: [<FsmState.DRIZZLE: 'drizzle'>] FsmState.TYPESCRIPT
Processing transition: [<FsmState.TYPESCRIPT: 'typescript'>] FsmState.HANDLER_TESTS
Processing transition: [<FsmState.HANDLER_TESTS: 'handler_tests'>] handlers
Processing transition: [<FsmState.HANDLERS: 'handlers'>] FsmState.COMPLETE


In [15]:
final_res

{'capabilities': [],
 'typespec_schema': <fsm_core.typespec.Success at 0xffff9aba5160>,
 'drizzle_schema': <fsm_core.drizzle.Success at 0xffffac1d5e80>,
 'typescript_schema': <fsm_core.typescript.Success at 0xffffa44524e0>,
 'handler_tests': {'updateUserName': <fsm_core.handler_tests.Success at 0xffffa4453740>,
  'greetUser': <fsm_core.handler_tests.Success at 0xffff9ab8fdd0>,
  'saveUserName': <fsm_core.handler_tests.Success at 0xffffa4687320>},
 'handlers': {'updateUserName': <fsm_core.handlers.Success at 0xffffa65e0140>,
  'saveUserName': <fsm_core.handlers.Success at 0xffff9abc8f80>,
  'greetUser': <fsm_core.handlers.Success at 0xffffac26b3b0>}}